# Download data files

In [ ]:
#define function to download all data files using web scraping
def get_citi_bike_file_list():
    
    from bs4 import BeautifulSoup
    import requests
    url = 'https://s3.amazonaws.com/tripdata/'  # stripped off"index.html, as url with index.html returns soup object with no file information
    
    html = requests.get(url).text
    soup = BeautifulSoup(html, "xml")  # the url is returning an xml file
    files = soup.find_all("Key")
    file_list =[]
    for file in files:
        file_list.append(file.text)

    return(file_list)

In [ ]:
# download all zip files in "citibike_data" folder & unzip them

def get_citibike_data_files():

    import os
    import zipfile
    url = 'https://s3.amazonaws.com/tripdata/'

    # download/unzip data files
    for file in get_citi_bike_file_list(): # get file list
        file_url = url + file              # create complete web url address to download
        
        #download files
        with open(file, "wb") as f:
                response = requests.get(file_url)
                f.write(response.content)
                
        # unzip data files
        with zipfile.ZipFile(file, "r") as zip_ref:
            zip_ref.extractall("citibike_data") 
            
        # remove zipped file after unziping
        os.remove(file)

# Reading all data files using pandas and transforming to required format with pivot_tables

In [53]:
import glob
import pandas as pd
import numpy as np
from datetime import datetime


# defining dataframes to store all data



for yr in range(2013,2019+1):
    df = pd.DataFrame()
    df_user = pd.DataFrame()
    df_start_station = pd.DataFrame()
    df_end_station = pd.DataFrame()
    df_static = pd.DataFrame()
    df_bike = pd.DataFrame()
    
    start = datetime.now().strftime('%H:%M:%S')
    print(f"starting at {start}")



    search_string = f"citibike_data/*{yr}*.csv"
    for csv_file in glob.glob(search_string):
        print(f"loading data from {csv_file}")
        # dataframe to hold data from a particular csv file from citibike database
        df = pd.read_csv(csv_file, na_values = ['\\N', 'NULL'])  # csv files have few nulls as '\N and 'NULL
        
        # renaming columns as standard names to facilitate concat operations later. header names changed a bit starting 2017
        df.columns = ['tripduration', 'starttime', 'stoptime', 'start station id',
                               'start station name', 'start station latitude',
                               'start station longitude', 'end station id', 'end station name',
                               'end station latitude', 'end station longitude', 'bikeid', 'usertype',
                               'birth year', 'gender']
        
        # drop columns not needed for our analysis
        df.drop(['end station name','end station latitude', 'end station longitude'], axis =1, inplace =True) 
        
        # fill Nulls with their median values. (broadly speaking only 'birth year and a few end_station_id have nulls')
        df['birth year'].fillna(df['birth year'].median(), inplace=True) 


        # convert time filelds to datetime format and creating columns for 'year', 'month' and 'start hour'
        df[['starttime', 'stoptime']] = df[['starttime', 'stoptime']].apply(pd.to_datetime) # convert to datetime
        df['start_hour'] = pd.DatetimeIndex(df['starttime']).hour # new columns
        df['end_hour'] = pd.DatetimeIndex(df['stoptime']).hour # new columns
        df['month']=pd.DatetimeIndex(df['starttime']).month # new columns
        df['year']=pd.DatetimeIndex(df['starttime']).year # new column
        df.drop(['starttime', 'stoptime'], axis =1, inplace =True) # dropping as not needed for our analysis
    
    
        # make dataframe containing bikeid specific data
        df4 = df.pivot_table(index = ['year', 'month', 'bikeid'], values = ['usertype'], aggfunc = [len])
        df4.columns = ['trip_count']
#         print("bike done")

        # make dataframe containing unique station_id, name, lat, long
        df3 =df[['start station id',
               'start station name', 'start station latitude',
               'start station longitude']]
        df3.drop_duplicates('start station id',inplace =True, keep = 'last') # last station id parameters may be the most updated one
        df3.columns =['station_id', 'station_name', 'station_latitude','station_longitude']
#         print("static done")
        
        # make dataframe containing tripduration mean and trip counts filetered by station_id and start_hour
        df2_start_station = df.pivot_table(index = ['year', 'month','start station id', 'start_hour'], values = [ 'usertype'], 
                             aggfunc = {'usertype':len })
        df2_start_station.columns =['trip_count']
        df2_end_station = df.pivot_table(index = ['year', 'month','end station id', 'end_hour'], values = ['usertype'], 
                             aggfunc = {'usertype':len })
        df2_end_station.columns =['trip_count']
#         print("stations done")

        # make dataframe containing tripduration mean and trip counts filetered by month/year, gender, birth_year
        df1= df.pivot_table(index = ['year', 'month','usertype','gender', 'birth year'], 
                            values = ['tripduration'], 
                            aggfunc = {'tripduration':[len, np.mean]} )
        df1.columns = ['trip_count', 'avg_duration']
        print("users done)")



        # appending dataframes that contain yearly filtered data   
        df_user = pd.concat([df_user, df1])
        df_start_station = pd.concat([df_start_station, df2_start_station])
        df_end_station = pd.concat([df_end_station, df2_end_station])
        df_static = pd.concat([df_static, df3])
        df_bike = pd.concat([df_bike, df4])
    
    
    # saving yearly dataframes as csv files (to be merged later)
    csv_user = f"filtered_data/user_data{yr}.csv"
    csv_start_station = f"filtered_data/station_start{yr}.csv"
    csv_end_station = f"filtered_data/station_end{yr}.csv"
    csv_static = f"filtered_data/static_data{yr}.csv"
    csv_bike = f"filtered_data/bike_data{yr}.csv"
    
    df_user.to_csv(csv_user)
    df_start_station.to_csv(csv_start_station)
    df_end_station.to_csv(csv_end_station)
    
    # static file to contain contain only on erecord for each station, hence drop duplicate.
    # in case of errors in lat/long for few records, we keep the last record, assuming  that 
    # last station id parameters may be the most updated one
    
    # this step to be repeated later while merging yearly files.
   
    df_static.drop_duplicates('station_id',inplace =True, keep = 'last') 
    df_static.set_index('station_id', drop = True, inplace = True)
    df_static.to_csv(csv_static)
    
    df_bike.to_csv(csv_bike)
                            
    # check shapes of our dataframe
    print(f"{yr} completed..")
#     df_user.shape, df_start_station.shape, df_end_station.shape, df_bike.shape, df_static.shape

        

starting at 11:27:09
loading data from citibike_data\2013-07 - Citi Bike trip data.csv


C:\Users\avise\Anaconda3\lib\site-packages\ipykernel_launcher.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


users done)
loading data from citibike_data\2013-08 - Citi Bike trip data.csv
users done)
loading data from citibike_data\2013-09 - Citi Bike trip data.csv
users done)
loading data from citibike_data\2013-10 - Citi Bike trip data.csv
users done)
loading data from citibike_data\2013-11 - Citi Bike trip data.csv
users done)
loading data from citibike_data\2013-12 - Citi Bike trip data.csv
users done)
loading data from citibike_data\201306-citibike-tripdata.csv
users done)
2013 completed..
starting at 11:27:42
loading data from citibike_data\2014-01 - Citi Bike trip data.csv
users done)
loading data from citibike_data\2014-02 - Citi Bike trip data.csv
users done)
loading data from citibike_data\2014-03 - Citi Bike trip data.csv
users done)
loading data from citibike_data\2014-04 - Citi Bike trip data.csv
users done)
loading data from citibike_data\2014-05 - Citi Bike trip data.csv
users done)
loading data from citibike_data\2014-06 - Citi Bike trip data.csv
users done)
loading data from c

In [54]:
df_user.trip_count.sum(), df_bike.trip_count.sum()

(8081216.0, 8081216)

In [97]:
# merge yearly files into combined dataframes 

import pandas as pd
import glob

# we will create 5 dataframes containing specific data from respective yearly files
df_user = pd.DataFrame()
df_start_station = pd.DataFrame()
df_end_station = pd.DataFrame()
df_static = pd.DataFrame()
df_bike = pd.DataFrame()

file_types =['user', 'bike', 'station_start','station_end', 'static'] # keyword used for different type of yearly csv files
for file_type in file_types:
    search_string = f"filtered_data/*{file_type}*.csv"
    
    # merge yearly files of specific types (we also need to take care of character '\N' appearing as null in csv files)
    for csv_file in glob.glob(search_string):
        if file_type == 'user':
            dfs = pd.read_csv(csv_file)
            df_user = pd.concat([df_user,dfs])
        elif file_type == 'station_start':
            dfs = pd.read_csv(csv_file)
            df_start_station = pd.concat([df_start_station,dfs])
        elif file_type == 'station_end':
            dfs = pd.read_csv(csv_file)
            df_end_station = pd.concat([df_end_station,dfs])
        elif file_type == 'bike':
            dfs = pd.read_csv(csv_file)
            df_bike = pd.concat([df_bike,dfs])
        elif file_type == 'static':
            dfs = pd.read_csv(csv_file)
            df_static = pd.concat([df_static,dfs])
            df_static.drop_duplicates('station_id',inplace =True, keep = 'last') 
            

            
df_user.shape, df_start_station.shape, df_end_station.shape, df_bike.shape, df_static.shape

((25849, 7), (923006, 5), (929566, 5), (652273, 4), (1019, 4))

In [98]:
df_user.dtypes, df_user.year.value_counts()

(year              int64
 month             int64
 usertype         object
 gender            int64
 birth year      float64
 trip_count      float64
 avg_duration    float64
 dtype: object, 2018    6848
 2017    5900
 2016    4248
 2019    3590
 2015    2420
 2014    1784
 2013    1059
 Name: year, dtype: int64)

In [99]:
# remove 2013 and 2019 records as they are for part of the year and we will not be using in our analysis
df_user = df_user[(df_user.year > 2013) & (df_user.year <2019)]
df_start_station = df_start_station[(df_start_station.year > 2013) & (df_start_station.year <2019)]
df_end_station = df_end_station[(df_end_station.year > 2013) & (df_end_station.year <2019)]
df_bike = df_bike[(df_bike.year > 2013) & (df_bike.year <2019)]
print(df_user.year.value_counts())
print(df_user.shape, df_start_station.shape, df_end_station.shape, df_bike.shape, df_static.shape)

2018    6848
2017    5900
2016    4248
2015    2420
2014    1784
Name: year, dtype: int64
(21200, 7) (756511, 5) (762193, 5) (528062, 4) (1019, 4)


In [100]:
df_static.nunique()

station_id           1018
station_name         1013
station_latitude     1015
station_longitude    1016
dtype: int64

In [101]:
# drop duplicate station records in static dataframe
df_static.drop_duplicates('station_id',inplace =True, keep = 'last')
print(df_user.shape, df_start_station.shape, df_end_station.shape, df_bike.shape, df_static.shape)

(21200, 7) (756511, 5) (762193, 5) (528062, 4) (1019, 4)


In [102]:
df_static.nunique()

station_id           1018
station_name         1013
station_latitude     1015
station_longitude    1016
dtype: int64

In [103]:
# check shapes of our dataframe


# writing final csv files to be used for 2014-18 analysis with Tableau Public
csv_user = f"Summary/user_data_all.csv"
csv_start_station = f"Summary/station_start_all.csv"
csv_end_station = f"Summary/station_end_all.csv"
csv_static = f"Summary/static_data_all.csv"
csv_bike = f"Summary/bike_data_all.csv"

df_user.to_csv(csv_user)
df_start_station.to_csv(csv_start_station)
df_end_station.to_csv(csv_end_station)
df_static.to_csv(csv_static)
df_bike.to_csv(csv_bike)

# making 2 new reformatted files 
####(new file format is and afterthought after running above code, hence to be merged later to above sections)

In [104]:
# making relformatted trips file
import glob
import pandas as pd
import numpy as np
from datetime import datetime


# defining dataframes to store all data



for yr in range(2014,2019+1):
    df = pd.DataFrame()
    df_trips = pd.DataFrame()
    

    search_string = f"citibike_data/*{yr}*.csv"
    for csv_file in glob.glob(search_string):
        print(f"loading data from {csv_file}")
        # dataframe to hold data from a particular csv file from citibike database
        df = pd.read_csv(csv_file, na_values = ['\\N', 'NULL'])  # csv files have few nulls as '\N and 'NULL
        
        # renaming columns as standard names to facilitate concat operations later. header names changed a bit starting 2017
        df.columns = ['tripduration', 'starttime', 'stoptime', 'start station id',
                               'start station name', 'start station latitude',
                               'start station longitude', 'end station id', 'end station name',
                               'end station latitude', 'end station longitude', 'bikeid', 'usertype',
                               'birth year', 'gender']
        
        # convert time filelds to datetime format and creating columns for 'year', 'month' and 'start hour'
        df['starttime'] = df['starttime'].apply(pd.to_datetime) # convert to datetime
        df['year']=pd.DatetimeIndex(df['starttime']).year # new column
        
        # drop columns not needed for our analysis
        df.drop(['starttime', 'stoptime', 'start station name', 'start station latitude',
                               'start station longitude','end station name','end station latitude', 'end station longitude', 
                                'bikeid', 'usertype', 'birth year', 'gender', 'starttime', 'stoptime'], 
                                axis =1, inplace =True) 
        

    

        # make dataframe containing tripduration mean and trip counts 
        df5 = df.pivot_table(index = ['year', 'start station id', 'end station id'], values = ['tripduration'], 
                            aggfunc = {'tripduration':[len, np.mean]} )
        
        print("trips done")



        # appending dataframes that contain yearly filtered data   
        df_trips = pd.concat([df_trips, df5])
       
    
    
    # saving yearly dataframes as csv files (to be merged later)
    csv_trips = f"filtered_data/trips_data{yr}.csv"
    df_trips.to_csv(csv_trips)
                          
    # check shapes of our dataframe
    print(f"{yr} completed..")
#     df_user.shape, df_start_station.shape, df_end_station.shape, df_bike.shape, df_static.shape

search_string = f"filtered_data/*trips*.csv"

df_trips = pd.DataFrame()
# merge yearly files of specific types (we also need to take care of character '\N' appearing as null in csv files)
for csv_file in glob.glob(search_string):
    dfs = pd.read_csv(csv_file)
    df_trips = pd.concat([df_trips,dfs])

csv_trips = f"Summary/trips_data_all.csv"

df_trips.to_csv(csv_trips)
   

loading data from citibike_data\2014-01 - Citi Bike trip data.csv
trips done
loading data from citibike_data\2014-02 - Citi Bike trip data.csv
trips done
loading data from citibike_data\2014-03 - Citi Bike trip data.csv
trips done
loading data from citibike_data\2014-04 - Citi Bike trip data.csv
trips done
loading data from citibike_data\2014-05 - Citi Bike trip data.csv
trips done
loading data from citibike_data\2014-06 - Citi Bike trip data.csv
trips done
loading data from citibike_data\2014-07 - Citi Bike trip data.csv
trips done
loading data from citibike_data\2014-08 - Citi Bike trip data.csv
trips done
loading data from citibike_data\201409-citibike-tripdata.csv
trips done
loading data from citibike_data\201410-citibike-tripdata.csv
trips done
loading data from citibike_data\201411-citibike-tripdata.csv
trips done
loading data from citibike_data\201412-citibike-tripdata.csv
trips done
2014 completed..
loading data from citibike_data\201501-citibike-tripdata.csv
trips done
loading

trips done
loading data from citibike_data\JC-201906-citibike-tripdata.csv
trips done
2019 completed..


C:\Users\avise\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (0,1,2,3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Reformat df_trips to contain lat/long for both start and end stations

In [100]:
import pandas as pd
df_trips = pd.read_csv("Summary/trips_data_all.csv")
df_trips.columns = ['serial_no', 'year', 'start_station_id', 'end_station_id', 'trips', 'mean_tripduration']
df_trips.drop(['serial_no'], axis = 1, inplace =True)
df_trips.drop([0,1], inplace = True)
df_trips.set_index('year', drop =True, inplace = True)
df_trips.head()

,start_station_id,end_station_id,trips,mean_tripduration
year,,,,
2014,72,72,6.0,1325.6666666666667
2014,72,79,3.0,1532.0
2014,72,116,3.0,991.3333333333334
2014,72,127,2.0,856.0
2014,72,128,1.0,1016.0


In [101]:
df_static = pd.read_csv("Summary/static_data_all.csv")

In [102]:
df_trips.head()
df_trips.dtypes

start_station_id     object
end_station_id       object
trips                object
mean_tripduration    object
dtype: object

In [103]:
# df_trips['start_station_id'] = df_trips['start_station_id'].astype('int')
df_trips.isnull().sum()
df_trips.dropna(inplace = True)


In [104]:
#df_trips['start_station_id'] = df_trips['start_station_id'].astype('float').astype('int64')
df_trips = df_trips.astype('float').astype('int64')

In [105]:
df_trips.dtypes

start_station_id     int64
end_station_id       int64
trips                int64
mean_tripduration    int64
dtype: object

In [106]:
# # create a look up dictionary for each station-id's lat/long
# stations = df_static.station_id.tolist()
# lats = df_static.station_latitude.tolist()
# longs = df_static.station_longitude.tolist()
# geoDict = {}
# for n in range(len(stations)):
#     geoDict.update({stations[n]:{'lat':lats[n],'long':longs[n]}}) 

In [107]:
df = df_trips.merge(df_static, how = 'left', left_on = 'start_station_id', right_on = 'station_id')
df.head()
df.shape

(7600684, 9)

In [108]:
df.head()

,start_station_id,end_station_id,trips,mean_tripduration,Unnamed: 0,station_id,station_name,station_latitude,station_longitude
0,72,72,6,1325,566,72.0,W 52 St & 11 Ave,40.767272,-73.993929
1,72,79,3,1532,566,72.0,W 52 St & 11 Ave,40.767272,-73.993929
2,72,116,3,991,566,72.0,W 52 St & 11 Ave,40.767272,-73.993929
3,72,127,2,856,566,72.0,W 52 St & 11 Ave,40.767272,-73.993929
4,72,128,1,1016,566,72.0,W 52 St & 11 Ave,40.767272,-73.993929


In [109]:
df = df.merge(df_static, how = 'left', left_on = 'end_station_id', right_on = 'station_id',suffixes = ('_start', '_end'))
df.head()
df.shape

(7600684, 14)

In [110]:
df.head()

,start_station_id,end_station_id,trips,mean_tripduration,Unnamed: 0_start,station_id_start,station_name_start,station_latitude_start,station_longitude_start,Unnamed: 0_end,station_id_end,station_name_end,station_latitude_end,station_longitude_end
0,72,72,6,1325,566,72.0,W 52 St & 11 Ave,40.767272,-73.993929,566.0,72.0,W 52 St & 11 Ave,40.767272,-73.993929
1,72,79,3,1532,566,72.0,W 52 St & 11 Ave,40.767272,-73.993929,175.0,79.0,Franklin St & W Broadway,40.719116,-74.006667
2,72,116,3,991,566,72.0,W 52 St & 11 Ave,40.767272,-73.993929,626.0,116.0,W 17 St & 8 Ave,40.741776,-74.001497
3,72,127,2,856,566,72.0,W 52 St & 11 Ave,40.767272,-73.993929,562.0,127.0,Barrow St & Hudson St,40.731724,-74.006744
4,72,128,1,1016,566,72.0,W 52 St & 11 Ave,40.767272,-73.993929,730.0,128.0,MacDougal St & Prince St,40.727103,-74.002971


In [111]:
df.columns

Index(['start_station_id', 'end_station_id', 'trips', 'mean_tripduration',
       'Unnamed: 0_start', 'station_id_start', 'station_name_start',
       'station_latitude_start', 'station_longitude_start', 'Unnamed: 0_end',
       'station_id_end', 'station_name_end', 'station_latitude_end',
       'station_longitude_end'],
      dtype='object')

In [112]:
df.drop(['Unnamed: 0_start','Unnamed: 0_end', 'station_id_start', 'station_id_end'], axis = 1, inplace = True)


In [113]:
df.head()

,start_station_id,end_station_id,trips,mean_tripduration,station_name_start,station_latitude_start,station_longitude_start,station_name_end,station_latitude_end,station_longitude_end
0,72,72,6,1325,W 52 St & 11 Ave,40.767272,-73.993929,W 52 St & 11 Ave,40.767272,-73.993929
1,72,79,3,1532,W 52 St & 11 Ave,40.767272,-73.993929,Franklin St & W Broadway,40.719116,-74.006667
2,72,116,3,991,W 52 St & 11 Ave,40.767272,-73.993929,W 17 St & 8 Ave,40.741776,-74.001497
3,72,127,2,856,W 52 St & 11 Ave,40.767272,-73.993929,Barrow St & Hudson St,40.731724,-74.006744
4,72,128,1,1016,W 52 St & 11 Ave,40.767272,-73.993929,MacDougal St & Prince St,40.727103,-74.002971


In [115]:
df.to_csv("Summary/trips_latlong_all.csv")

In [117]:
import glob
import pandas as pd
import numpy as np
from datetime import datetime


# defining dataframes to store all data



for yr in range(2014,2018+1):
    df = pd.DataFrame()
    df_bike = pd.DataFrame()
    
    start = datetime.now().strftime('%H:%M:%S')
    print(f"starting at {start}")



    search_string = f"citibike_data/*{yr}*.csv"
    for csv_file in glob.glob(search_string):
        print(f"loading data from {csv_file}")
        # dataframe to hold data from a particular csv file from citibike database
        df = pd.read_csv(csv_file, na_values = ['\\N', 'NULL'])  # csv files have few nulls as '\N and 'NULL
        
        # renaming columns as standard names to facilitate concat operations later. header names changed a bit starting 2017
        df.columns = ['tripduration', 'starttime', 'stoptime', 'start station id',
                               'start station name', 'start station latitude',
                               'start station longitude', 'end station id', 'end station name',
                               'end station latitude', 'end station longitude', 'bikeid', 'usertype',
                               'birth year', 'gender']
        
        # drop columns not needed for our analysis
        df.drop(['end station name','end station latitude', 'end station longitude'], axis =1, inplace =True) 
        
        # fill Nulls with their median values. (broadly speaking only 'birth year and a few end_station_id have nulls')
        df['birth year'].fillna(df['birth year'].median(), inplace=True) 


        # convert time filelds to datetime format and creating columns for 'year', 'month' and 'start hour'
        df[['starttime', 'stoptime']] = df[['starttime', 'stoptime']].apply(pd.to_datetime) # convert to datetime
        df['start_hour'] = pd.DatetimeIndex(df['starttime']).hour # new columns
        df['end_hour'] = pd.DatetimeIndex(df['stoptime']).hour # new columns
        df['month']=pd.DatetimeIndex(df['starttime']).month # new columns
        df['year']=pd.DatetimeIndex(df['starttime']).year # new column
        df.drop(['starttime', 'stoptime'], axis =1, inplace =True) # dropping as not needed for our analysis
    
    
        # make dataframe containing bikeid specific data
        df4 = df.pivot_table(index = ['year', 'month', 'start_hour','bikeid'], values = ['usertype'], aggfunc = [len])
        df4.columns = ['trip_count']
#         print("bike done")

        


        # appending dataframes that contain yearly filtered data   
        df_bike = pd.concat([df_bike, df4])
    
    
    # saving yearly dataframes as csv files (to be merged later)
#     csv_bike = f"filtered_data/bike_data{yr}.csv"
    

    
#     df_bike.to_csv(csv_bike)
                            
    # check shapes of our dataframe
    print(f"{yr} completed..")
df_bike.shape


starting at 21:32:03
loading data from citibike_data\2014-01 - Citi Bike trip data.csv
loading data from citibike_data\2014-02 - Citi Bike trip data.csv
loading data from citibike_data\2014-03 - Citi Bike trip data.csv
loading data from citibike_data\2014-04 - Citi Bike trip data.csv
loading data from citibike_data\2014-05 - Citi Bike trip data.csv
loading data from citibike_data\2014-06 - Citi Bike trip data.csv
loading data from citibike_data\2014-07 - Citi Bike trip data.csv
loading data from citibike_data\2014-08 - Citi Bike trip data.csv
loading data from citibike_data\201409-citibike-tripdata.csv
loading data from citibike_data\201410-citibike-tripdata.csv
loading data from citibike_data\201411-citibike-tripdata.csv
loading data from citibike_data\201412-citibike-tripdata.csv
2014 completed..
starting at 21:44:12
loading data from citibike_data\201501-citibike-tripdata.csv
loading data from citibike_data\201502-citibike-tripdata.csv
loading data from citibike_data\201503-citibike

(2642158, 1)

In [118]:
df_bike.head()

trip_count
year month start_hour bikeid            
2018 1     0          14539            2
                      14545            1
                      14556            1
                      14558            2
                      14566            1

In [119]:
df_bike.to_csv("Summary/bike_data_all_v2.csv")

#### Note: Few anamolies have been filtered and dealt with whiel analyzing in tableau e.g.
    1. Birth years reporting (above 80 years of age is considered wrong reporting/error)
    2. two co-ordinates seen far away (in other state)

https://www.olgatsubiks.com/single-post/2017/03/20/How-to-save-Tableau-Public-workbooks-privately-on-your-computer
https://onlinehelp.tableau.com/current/pro/desktop/en-us/embed.htm